In [18]:
import tensorflow as tf
import numpy as np
from keras.datasets import mnist

In [19]:
"""Load data"""

(X_train, y_train), (X_test, y_test) = mnist.load_data(path="mnist.npz")
num_classes = 10 # 0 through 9

# One-hot encode y_train and y_test
y_train = np.array([np.array([0]*(y)+[1]+[0]*(num_classes-y-1)) for y in y_train])
y_test  = np.array([np.array([0]*(y)+[1]+[0]*(num_classes-y-1)) for y in y_test])

# Flatten Xs
X_train = np.array([X.flatten() for X in X_train])
X_test  = np.array([X.flatten() for X in X_test])

# Create placeholders
input_size = X_train.shape[1]
X = tf.placeholder("float", [None, input_size])
y = tf.placeholder("float", [None, num_classes])

In [20]:
"""Hyperparams"""

BATCH_SIZE = 100
EPOCHS = 15
LEARNING_RATE = 0.001
DROPOUT = 0.5 # Only used for ConvNets
ACC_EVERY = 1 # Print accuracy every __

In [21]:
"""Utility functions"""

activation = tf.nn.relu

def init_weights(weights_sizes):
    return [tf.Variable(tf.random_normal(weight_size))
           for weight_size in weights_sizes]

def init_biases(biases_sizes):
    return [tf.Variable(tf.random_normal([bias_size])) for bias_size in biases_sizes]

def fully_connected(prev_layer, weight, bias, activation=activation):
    layer = tf.add(tf.matmul(prev_layer, weight), bias)
    layer = activation(layer)
    return layer

def convolution(X, W, b, stride=1):
    X = tf.nn.conv2d(X, W, strides=[1, stride, stride, 1], padding='SAME')
    X = tf.nn.bias_add(X, b)
    return activation(X)

def max_pool(X):
    return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
"""Multilayer perceptron"""
# Hooray, generalized!
# https://www.tensorflow.org used as reference
# https://github.com/aymericdamien/TensorFlow-Examples was also useful

layers_sizes = [255, 255]

weights_sizes = zip([input_size]+layers_sizes, layers_sizes+[num_classes])
biases_sizes = layers_sizes+[num_classes]
weights = init_weights(weights_sizes)
biases = init_biases(biases_sizes)


def mlp_layers():
    layers = []
    for ind, layer_size in enumerate(layers_sizes):
        try:
            last_layer = layers[-1]
        except IndexError:
            last_layer = X
        layers.append(fully_connected(last_layer, weights[ind], biases[ind]))
    layers.append(tf.add(tf.matmul(layers[-1], weights[-1]), biases[-1]))
    return layers

predictor = mlp_layers()[-1]
x_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictor, labels=y))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(x_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for epoch in range(EPOCHS):
    for step in range(len(X_train)//BATCH_SIZE):
        start_ind = (step*BATCH_SIZE) % len(X_train)
        end_ind = ((step+1)*BATCH_SIZE) % len(X_train)
        batch_X, batch_y = X_train[start_ind:end_ind], y_train[start_ind:end_ind]
        sess.run(train_step, feed_dict={X:batch_X, y:batch_y})
    
    if epoch%ACC_EVERY == 0:
        correct_prediction = tf.equal(tf.argmax(predictor, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print "Epoch ", str(epoch).zfill(3), " | Testing Acc:", accuracy.eval({X: X_test, y: y_test})

print "Final Accuracy:", accuracy.eval({X: X_test, y: y_test})

Epoch  000  | Testing Acc: 0.8662
Epoch  001  | Testing Acc: 0.8991
Epoch  002  | Testing Acc: 0.9142
Epoch  003  | Testing Acc: 0.9193
Epoch  004  | Testing Acc: 0.9242
Epoch  005  | Testing Acc: 0.9274
Epoch  006  | Testing Acc: 0.933
Epoch  007  | Testing Acc: 0.9338
Epoch  008  | Testing Acc: 0.9364
Epoch  009  | Testing Acc: 0.9381
Epoch  010  | Testing Acc: 0.94
Epoch  011  | Testing Acc: 0.9393
Epoch  012  | Testing Acc: 0.9432
Epoch  013  | Testing Acc: 0.942
Epoch  014  | Testing Acc: 0.9389
Final Accuracy: 0.9389


In [22]:
"""
CONVOLUTIONAL NEURAL NETWORKS
"""
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

CONV_BATCH_SIZE = 5000 # too slow otherwise
dropout_prob = tf.placeholder(tf.float32)
layers_sizes = [(5,5,1,16), (5,5,16,32), (32*7**2,512),(512,num_classes)]
num_conv_layers = 2
biases_sizes = [size[-1] for size in layers_sizes]
weights = init_weights(layers_sizes)
biases = init_biases(biases_sizes)

def conv_layers(X, weights, biases, dropout):
    X = tf.reshape(X, shape=[-1,28,28,1])
    layers = []
    # generate conv layers:
    for ind, layer_size in enumerate(weights[:num_conv_layers]):
        try:
            last_layer = layers[-1]
        except IndexError:
            last_layer = X
        layers.append(convolution(last_layer, weights[ind], biases[ind]))
        layers.append(max_pool(layers[-1]))
        
    # generate fully connected layers:
    for iter_ind, layer_size in enumerate(weights[num_conv_layers:-1]):
        try:
            last_layer = layers[-1]
        except IndexError:
            last_layer = X
        ind = iter_ind+num_conv_layers
        layer = tf.reshape(last_layer, [-1, weights[ind].get_shape().as_list()[0]])
        layer = tf.add(tf.matmul(layer ,weights[ind]), biases[ind])
        layer = tf.nn.dropout(layer, dropout)
        layers.append(activation(layer))
    
    last = tf.add(tf.matmul(layers[-1], weights[-1]), biases[-1])
    layers.append(last)
    return layers

predictor = conv_layers(X, weights, biases, dropout_prob)[-1]
x_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictor, labels=y))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(x_entropy)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for epoch in range(EPOCHS):
    for step in tqdm(range(len(X_train)//CONV_BATCH_SIZE)):  
        start_ind = (step*CONV_BATCH_SIZE) % len(X_train)
        end_ind = ((step+1)*CONV_BATCH_SIZE) % len(X_train)
        batch_X, batch_y = X_train[start_ind:end_ind], y_train[start_ind:end_ind]
        sess.run(train_step, feed_dict={X:batch_X, y:batch_y, dropout_prob: DROPOUT})
    print("ended stepping")
    
    if epoch % ACC_EVERY == 0:
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print "Epoch ", str(epoch).zfill(3), " | Testing Acc:", accuracy.eval({X: X_test, y: y_test, dropout_prob: DROPOUT})
    
print "Final Accuracy:", accuracy.eval({X: X_test, y: y_test})

100%|██████████| 12/12 [01:14<00:00,  6.24s/it]


ended stepping
Epoch  000  | Testing Acc:

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder', defined at:
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/